In [1]:
setwd('../')

#### Setup for severe - non-severe patients

In [ ]:
phenotype <- "severe" # "severe" for severe-only analysis, "notsevere" for non-severe genes.

destfile= "Plots/Figure4_" #Directory destination and name of the csv and pdf files that will be retrieved as output.

 
#libraries needed:
library("AnnotationDbi")
library("AnnotationDbi")
library("org.Hs.eg.db")
library("igraph")

Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain 

### Read Community structure output files from MolTi

In [ ]:
cero <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution0MultilayerNetwork5112018.csv")
cero <- strsplit(cero,"\t")
cerocinco <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution05MultilayerNetwork5112018.csv")
cerocinco <- strsplit(cerocinco,"\t")
uno <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution1MultilayerNetwork5112018.csv")
uno <- strsplit(uno,"\t")
unocinco <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution15MultilayerNetwork5112018.csv")
unocinco <- strsplit(unocinco,"\t")
dos <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution2MultilayerNetwork5112018.csv")
dos <- strsplit(dos,"\t")
doscinco <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution25MultilayerNetwork5112018.csv")
doscinco <- strsplit(doscinco,"\t")
tres <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution3MultilayerNetwork5112018.csv")
tres <- strsplit(tres,"\t")
trescinco <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution35MultilayerNetwork5112018.csv")
trescinco <- strsplit(trescinco,"\t")
cuatro <- readLines("data/MolTi/Community_Analysis/CommunitiesResolution4MultilayerNetwork5112018.csv")
cuatro <- strsplit(cuatro,"\t")

In [ ]:
lista_comunidades <- list(cero,cerocinco,uno,unocinco,dos,doscinco,tres,trescinco,cuatro)

#### Load  severe diagnosis for each patient

In [ ]:
patients2 <- read.table(file= "data/MolTi/Community_Analysis/originalSevereDiagnosis.csv",sep= ",",header= F)
patients2[,1] <- as.character(patients2[,1])
patients2[,2] <- as.character(patients2[,2])
patients2 <- cbind(patients2,0)
patients2[,3] <- sample(patients2[,2])

In [ ]:
head(patients2) # Column '0' are the randomized assignments

### Retrieve specific Compound Heterozygous Mutations for Severe & Non-severe groups

In [ ]:
#Compound Heterozygous mutations
patients <- read.table(file= "data/InputGenes/compound_all_nov2018.csv")
patients[,1] <- as.character(patients[,1])
patients[,2] <- as.character(patients[,2])
patients <- split(patients[,2],patients[,1])
#severepatients <- patients[which(patients2[,3]=="severe")] #Uncomment and comment the next one for randomizing
severepatients <- patients[patients2[,1][which(patients2[,2]=="severe")]] ##Uncomment and comment the previous one for the original plot
#notseverepatients <- patients[which(patients2[,3]=="notsevere")] #Uncomment and comment the next one for randomizing
notseverepatients <- patients[patients2[,1][which(patients2[,2]=="notsevere")]] #Uncomment and comment the previous one for the original plot
severe<- unlist(severepatients)
names(severe) <- NULL
notsevere <- unlist(notseverepatients)
names(notsevere) <- NULL
severeonly <- setdiff(severe,notsevere)
notsevereonly <- setdiff(notsevere,severe)

severepatientsnames <- names(severepatients)
notseverepatientsnames <- names(notseverepatients)

In [ ]:
head(patients)

### Retrieve specific Copy Number Variations for Severe & Non-severe groups

In [ ]:
cnvdata <- read.csv("data/InputGenes/CNV_genes_for_iker.tsv",sep= "\t")
cnvdata[,"gene"] <- as.character(cnvdata[,"gene"])
genis <- c(cnvdata["gene"])$gene
genis <- mapIds(org.Hs.eg.db,keys = genis,column = "SYMBOL",keytype="ENSEMBL",multiVals = "first")
# Manual mapping for missing IDs
genis["ENSG00000005955"] <- "GGNBP2" 
genis["ENSG00000006114"] <- "SYNGR"
genis["ENSG00000108264"] <- "TADA2A"
genis["ENSG00000108270"] <- "AATF"
genis["ENSG00000108272"] <- "DHRS11"
genis["ENSG00000108278"] <- "ZNHI3"
genis["ENSG00000108753"] <- "HNF1B"
genis["ENSG00000129282"] <- "MRM1"
genis["ENSG00000132130"] <- "LHX1"
genis["ENSG00000141140"] <- "MYO19"
genis["ENSG00000141141"] <- "DDX52"
genis["ENSG00000161326"] <- "DUSP14"
genis["ENSG00000167230"] <- "C17orf78"
genis["ENSG00000174093"] <- "RP11-1407O15.2"
genis["ENSG00000184886"] <- "PIGW"
genis["ENSG00000197681"] <- "TBC1D3"
genis["ENSG00000203815"] <- "FAM231D"
genis["ENSG00000219492"] <- "RP11-1396O31.13"
genis["ENSG00000229924"] <- "FAM90A26"
genis["ENSG00000250913"] <- "USP17L23"
genis["ENSG00000268172"] <- "AL590452.1"
genis <- as.matrix(genis)
rownames(genis) <- NULL
genis <- genis[,1]
cnvdata <- cbind(cnvdata,genis)
severedata <- cnvdata[,c("gene","whole_gene",severepatientsnames,"genis")]
milddata <- cnvdata[,c("gene","whole_gene",notseverepatientsnames,"genis")]

In [ ]:
severedata <- severedata[apply(X = severedata,MARGIN = 1,function(x) paste0(x[3:10],collapse='_')) != '2_2_2_2_2_2_2_2',]
head(severedata)

In [ ]:
milddata <- milddata[apply(X = milddata,MARGIN = 1,function(x) paste0(x[3:10],collapse='_')) != '2_2_2_2_2_2_2_2',]
head(milddata)

In [ ]:
rownames(severedata) <- NULL
rownames(milddata) <- NULL
severecnvs <- unique(as.character(severedata[,"genis"]))
mildcnvs <- unique(as.character(milddata[,"genis"]))
cnvsevereonly <- setdiff(severecnvs,mildcnvs)
cnvsnotsevereonly <- setdiff(mildcnvs,severecnvs)

In [ ]:
cnvsevereonly

### Load CMS Causal genes

In [ ]:
cosa <- read.csv("data/InputGenes/cmsgenes.csv",header=FALSE) #http://www.musclegenetable.fr/4DACTION/Blob_groupe2
#cosa <- read.csv("data/InputGenes/genespaper.csv",header=FALSE) #Table PMID:30552423
cosa <- cosa[,1]
cosa <- as.character(cosa)
cosa

In [ ]:
severeonly2 <- data.frame(severeonly)
cosa2 <- data.frame(cosa)
cnvsevereonly2 <- data.frame(cnvsevereonly)
notsevereonly2 <- data.frame(notsevereonly)
notcnvsevereonly2 <- data.frame(cnvsnotsevereonly)


severeonly2[,2] <- 'CHVsevereonly'
cosa2[,2] <- 'causal'
cnvsevereonly2[,2] <- 'CNVsevereonly'
notsevereonly2[,2] <- 'CHVnotsevereonly'
notcnvsevereonly2[,2] <- 'CNVsnotsevereonly'

#For Cytoscape plots
write.table(rbind(as.matrix(severeonly2),as.matrix(cnvsevereonly2),as.matrix(cosa2)),file='Cytoscape_Session/SevereMetadata_Figure4_Supp_Fig_6.csv',sep='csv',quote=F,row.names = F,col.names = F)
write.table(rbind(as.matrix(notsevereonly2),as.matrix(notcnvsevereonly2),as.matrix(cosa2)),file='Cytoscape_Session/NOTSevereMetadata_Figure4_Supp_Fig_6.csv',sep='csv',quote=F,row.names = F,col.names = F)


### Join causal and mutated genes

In [ ]:
# Setup depending on severity groups
if(phenotype == "severe"){
  algenes <- c(severeonly,cosa,cnvsevereonly) #Severe-Only Analysis 
}
if(phenotype == "notsevere"){
  algenes <- c(notsevereonly,cosa,cnvsnotsevereonly) #To get a Mild-Only  Analysis
}

In [ ]:
# Solve some missing gene mappings
if("KIAA1919" %in% algenes){
  algenes <- replace(algenes,algenes== "KIAA1919","MFSD4B")
}
if("FAM188B" %in% algenes){
  algenes <- replace(algenes,algenes== "FAM188B","MINDY4")
}
if("C14orf159" %in% algenes){
  algenes <- replace(algenes,algenes== "C14orf159","DGLUCY")
}
if("C1orf168" %in% algenes){
  algenes <- replace(algenes,algenes== "C1orf168","FYB2")
}
if("IKBKAP" %in% algenes){
  algenes <- replace(algenes,algenes== "IKBKAP","ELP1")
}
if("SYNGR" %in% algenes){
  algenes <- replace(algenes,algenes== "SYNGR","SYNGR1")
}
if("ZNHI3" %in% algenes){
  algenes <- replace(algenes,algenes== "ZNHI3","ZNHIT3")
}
if("FAM231D" %in% algenes){
  algenes <- replace(algenes,algenes== "FAM231D","LINC01145")
}
if("SELO" %in% algenes){
  algenes <- replace(algenes,algenes== "SELO","SELENOO")
}
if("MFSD7" %in% algenes){
  algenes <- replace(algenes,algenes== "MFSD7","SLC49A3")
}

#Convert to entrez
algenes2 <- algenes
algenes <- mapIds(org.Hs.eg.db,keys = algenes,column = "ENTREZID",keytype="SYMBOL",multiVals = "first")
names(algenes) <- NULL

### Create a table with the community ID of each gene in 'genescausales' vector.

In [ ]:
genescausales <- mapIds(org.Hs.eg.db,keys = algenes2,column = "ENTREZID",keytype="SYMBOL",multiVals = "first")
  genescausales <- as.matrix(genescausales)
  rownames(genescausales) <- NULL
  genescausales <- genescausales[,1]
  genescausales <- unlist(genescausales) #need this unlist as sometimes R does not create a c() vector with the previous line
  disease <- genescausales
  res <- matrix(nrow= length(disease),ncol= 9)
  rownames(res) <- disease

#Remove genes not present on Multilayer network
curl <- lapply(as.list(genescausales),function(x) which(sapply(cuatro, function(y) x %in% y)))
res <- res[!unlist(lapply(curl,function(x) length(x)==0)),]

In [ ]:
head(res)

In [ ]:
dim(res)

In [ ]:
for(i in 1:9){
  current <- unlist(lapply(as.list(rownames(res)),function(x) which(sapply(lista_comunidades[[i]], function(y) x %in% y))))
  res[,i] <- current
}

In [ ]:
head(res)

###  Generate Figure 4 graph (+ input graph for Cytoscape)

In [ ]:
library(parallelDist)
library(reshape2)

In [ ]:
melted_dist <- as.matrix(parallelDist(res,'hamming',threads = 7)*9)
diag(melted_dist) <- NA
melted_dist[lower.tri(melted_dist)] <- NA
head(melted_dist)

In [ ]:
rownames(melted_dist) <- mapIds(org.Hs.eg.db,keys = rownames(melted_dist),column = "SYMBOL",keytype="ENTREZID",multiVals = "first")
colnames(melted_dist) <- rownames(melted_dist)

In [ ]:
melted_dist <- melt(melted_dist)
write.table(melted_dist[!is.na(melted_dist[,3]),],file='data/Figure3_input.csv',row.names = F,col.names = T,quote=F,sep=',') #Input for Figure 3 plots in Cytoscape!
melted_dist <- melted_dist[melted_dist[,3]==0,] # Get pairs that are always together trough resolution
melted_dist <- na.omit(melted_dist)
dim(melted_dist)

In [ ]:
head(melted_dist)

In [ ]:
g=simplify(graph_from_edgelist(as.matrix(melted_dist[,1:2]),directed = F))
g

In [ ]:
components(g)

In [ ]:
layouts <- grep("^layout_", ls("package:igraph"), value=TRUE)[-1]
layouts

In [ ]:
#test.layout <- layout_components(g,weight.edge.lengths = edge_density(g)/100)
options(repr.plot.width=25, repr.plot.height=15)
set.seed(2020) #seed for layout
plot(g, layout = layout_components)

#### Write CSV file for plotting with Cytoscape (Figure 4)

In [ ]:
write.csv(x= as_edgelist(g),file= paste0(destfile,phenotype,".csv"),row.names=FALSE) 